## Encoding categorical features

Categorical features are non-numerical features with a limited amount of possible options.

Before feeding them into most ML algorithms we must convert them into numerical features using either:

* **Label encoding**: Each unique categor is assigned to a numerical value: 0, 1, 2, 3...
* **One-hot encoding** A new binary feature is created for each category.

In [13]:
import pandas as pd
import seaborn as sns

In [14]:
titanic = sns.load_dataset('titanic')
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [15]:
cat_columns = [col for col in titanic.columns if titanic[col].dtype in ['object', 'category']]
cat_columns

['sex', 'embarked', 'class', 'who', 'deck', 'embark_town', 'alive']

In [16]:
df_categories = titanic[cat_columns]
df_categories.head()

,sex,embarked,class,who,deck,embark_town,alive
0,male,S,Third,man,NaN,Southampton,no
1,female,C,First,woman,C,Cherbourg,yes
2,female,S,Third,woman,NaN,Southampton,yes
3,female,S,First,woman,C,Southampton,yes
4,male,S,Third,man,NaN,Southampton,no


In [17]:
for cat in cat_columns:
    print(f"{cat}: {df_categories[cat].unique()}")

sex: ['male' 'female']
embarked: ['S' 'C' 'Q' nan]
class: ['Third', 'First', 'Second']
Categories (3, object): ['First', 'Second', 'Third']
who: ['man' 'woman' 'child']
deck: [NaN, 'C', 'E', 'G', 'D', 'A', 'B', 'F']
Categories (7, object): ['A', 'B', 'C', 'D', 'E', 'F', 'G']
embark_town: ['Southampton' 'Cherbourg' 'Queenstown' nan]
alive: ['no' 'yes']


**Label encoding**

Label encoding maps each category  into a numerical value.

Use label encoding if:

* Categories have a natural order;
* There are only two categories;
* Using one-hot encoding leads to a large number of features

Manual mapping:

In [18]:
titanic['embarked'] = titanic['embarked'].map({'S': 0, 'C': 1, 'Q': 2})
print(titanic['embarked'].unique())

[ 0.  1.  2. nan]


Метод map в pandas используется для замены значений в столбце (или серии) на другие значения с использованием словаря, серии или функции. Он применяется к столбцам данных, чтобы выполнить преобразования значений в столбце согласно заданному соответствию.

Простейшее использование map выглядит следующим образом:
dataframe['column_name'] = dataframe['column_name'].map(mapping_dictionary)

Auto mapping using Label Encoder from Scikit-learn:

In [19]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

titanic['deck'] = le.fit_transform(titanic['deck'])
print(titanic['deck'].unique())

[7 2 4 6 3 0 1 5]


**One-hot encoding**

In one-hot encoding, a new binary feature is created for each category, and the value of this feature is set to 1 if the observation belongs to that category and 0 otherwise.

Use one-hot encoding if:

* Categories have no natural order;
* Number of categories is small (but not 2)

In [20]:
titanic = pd.get_dummies(data = titanic, columns = ["who"])

In [21]:
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,adult_male,deck,embark_town,alive,alone,who_child,who_man,who_woman
0,0,3,male,22.0,1,0,7.2500,0.0,Third,True,7,Southampton,no,False,False,True,False
1,1,1,female,38.0,1,0,71.2833,1.0,First,False,2,Cherbourg,yes,False,False,False,True
2,1,3,female,26.0,0,0,7.9250,0.0,Third,False,7,Southampton,yes,True,False,False,True
3,1,1,female,35.0,1,0,53.1000,0.0,First,False,2,Southampton,yes,False,False,False,True
4,0,3,male,35.0,0,0,8.0500,0.0,Third,True,7,Southampton,no,True,False,True,False


## Auto-map remaining non-numerical features

In [22]:
cat_columns = [col for col in titanic.columns if titanic[col].dtype in ['object', 'category', 'bool']]
cat_columns

['sex',
 'class',
 'adult_male',
 'embark_town',
 'alive',
 'alone',
 'who_child',
 'who_man',
 'who_woman']

In [23]:
for column in cat_columns:
    titanic[column] = le.fit_transform(titanic[column])

In [24]:
df_categories = titanic[cat_columns]
for cat in cat_columns:
    print(f"{cat}: {df_categories[cat].unique()}")

sex: [1 0]
class: [2 0 1]
adult_male: [1 0]
embark_town: [2 0 1 3]
alive: [0 1]
alone: [0 1]
who_child: [0 1]
who_man: [1 0]
who_woman: [0 1]


In [26]:
titanic.to_json("../TrainingData/titanic_encoded.json", orient = "records")